# Environment Sanity Check #

In [1]:
!nvidia-smi

Sat Sep 14 16:45:26 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:B3:00.0 Off |                    0 |
| N/A   30C    P0    54W / 300W |   1051MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [167]:
import cugraph
import cudf
import pandas as pd
import numpy as np
import networkx as nx

import plotly_express as px
%matplotlib inline

# 1. Connected Components

We first start by creating a list of edges along with the distances which we will add as the weight of the edge:


In [3]:
edgelist = [['Mannheim', 'Frankfurt', 85], ['Mannheim', 'Karlsruhe', 80], ['Erfurt', 'Wurzburg', 186], ['Munchen', 'Numberg', 167], ['Munchen', 'Augsburg', 84], ['Munchen', 'Kassel', 502], ['Numberg', 'Stuttgart', 183], ['Numberg', 'Wurzburg', 103], ['Numberg', 'Munchen', 167], ['Stuttgart', 'Numberg', 183], ['Augsburg', 'Munchen', 84], ['Augsburg', 'Karlsruhe', 250], ['Kassel', 'Munchen', 502], ['Kassel', 'Frankfurt', 173], ['Frankfurt', 'Mannheim', 85], ['Frankfurt', 'Wurzburg', 217], ['Frankfurt', 'Kassel', 173], ['Wurzburg', 'Numberg', 103], ['Wurzburg', 'Erfurt', 186], ['Wurzburg', 'Frankfurt', 217], ['Karlsruhe', 'Mannheim', 80], ['Karlsruhe', 'Augsburg', 250],["Mumbai", "Delhi",400],["Delhi", "Kolkata",500],["Kolkata", "Bangalore",600],["TX", "NY",1200],["ALB", "NY",800]]

Now we want to find out distinct continents and their cities from this graph.
First, we will need to create a cudf dataframe with edges in it. Right now I am creating a pandas dataframe and converting it to cudf dataframe but in a real-life scenario, we will read from a csv file of edges.

In [4]:
# create a pandas dataframe of edges
pandas_df = pd.DataFrame(edgelist)
pandas_df.columns = ['src','dst','distance']
# create a pandas dataframe of reversed edges as we have a undirected graph
rev_pandas_df = pandas_df.copy()
rev_pandas_df.columns = ['dst','src','distance']
rev_pandas_df = rev_pandas_df[['src','dst','distance']]
# concat all edges
pandas_df = pd.concat([pandas_df,rev_pandas_df])

Now our pandas df contains edges in both directions. And our node names in src and dst columns are in str format. Apparently, cuGraph doesn't like that and only works with integer node IDs.

In [5]:
# CuGraph works with only integer node IDS
unique_destinations = set()
for [src,dst,dis] in edgelist:
  unique_destinations.add(src)
  unique_destinations.add(dst)
    
# create a map of city and a unique id
city_id_dict = {}
for i, city in enumerate(unique_destinations):
  city_id_dict[city]=i
# create 2 columns that contain the integer IDs for src and dst
pandas_df['src_int'] = pandas_df['src'].apply(lambda x : city_id_dict[x])
pandas_df['dst_int'] = pandas_df['dst'].apply(lambda x : city_id_dict[x])

Now comes the main part that we should focus on:

In [6]:
cuda_g = cudf.DataFrame.from_pandas(pandas_df)
# cugraph needs node IDs to be int32 and weights to be float
cuda_g['src_int'] = cuda_g['src_int'].astype(np.int32)
cuda_g['dst_int'] = cuda_g['dst_int'].astype(np.int32)
cuda_g['distance'] = cuda_g['distance'].astype(np.float)
G = cugraph.Graph()
G.add_edge_list(cuda_g["src_int"],cuda_g["dst_int"] , cuda_g['distance'])
cugraph.strongly_connected_components(G)

,labels,vertices
0,0,0
1,1,1
2,1,2
3,1,3
4,0,4
5,5,5
6,5,6
7,1,7
8,0,8
9,1,9


# 2. Shortest Path

We already have our Graph as before. We can find the shortest distance from a source node to all nodes in the graph.

In [7]:
# get distances from source node 1
distances = cugraph.sssp(G, 1)
# filter infinite distances
distances = cugraph.traversal.filter_unreachable(distances)
distances

,vertex,distance,predecessor
1,1,0.0,-1
2,2,576.0,11
3,3,488.0,11
7,7,472.0,9
9,9,289.0,10
10,10,186.0,1
11,11,403.0,10
12,12,456.0,9
15,15,540.0,12
16,16,568.0,3


In [8]:
#Getting the path is as simple as:

# 1 to 15

path = []

dest = 15
while dest != 1:
   dest = distances[distances['vertex'] == dest]['predecessor'].values[0]
   path.append(dest)
print(path[::-1])

[1, 10, 9, 12]


# 3. Pagerank

In [9]:
# Loading the file as cudf
fb_cudf = cudf.read_csv("facebook_combined.txt", sep=' ', names=['src', 'dst'],dtype =['int32','int32'])

In [10]:
# adding reverse edges also
rev_fb_cudf = fb_cudf[['dst','src']]
rev_fb_cudf.columns = ['src','dst']
fb_cudf = cudf.concat([fb_cudf,rev_fb_cudf])

In [11]:
# creating the graph
fb_G = cugraph.Graph()
fb_G.add_edge_list(fb_cudf["src"],fb_cudf["dst"])

In [12]:
# Call cugraph.pagerank to get the pagerank scores
fb_pagerank = cugraph.pagerank(fb_G)

In [13]:
fb_pagerank.sort_values(by='pagerank',ascending=False).head()

,vertex,pagerank
3437,3437,0.007575
107,107,0.006888
1684,1684,0.006309
0,0,0.006225
1912,1912,0.003817


# 4. Link Prediction

In [14]:
max_vertex_id = fb_pagerank['vertex'].max()

In [15]:
max_vertex_id

4038

In [16]:
data = []
for x in range(0,max_vertex_id+1):
  for y in range(0,max_vertex_id+1):
    data.append([x,y])

In [17]:
cudf_nodes =cudf.from_pandas(pd.DataFrame(data))

cudf_nodes.columns = ['src','dst']

In [18]:
cudf_nodes['src'] = cudf_nodes['src'].astype(np.int32)
cudf_nodes['dst'] = cudf_nodes['dst'].astype(np.int32)

In [19]:
jaccard_coeff_between_nodes = cugraph.link_prediction.jaccard(fb_G,cudf_nodes["src"],cudf_nodes["dst"])

In [20]:
jaccard_coeff_between_nodes.head()

,source,destination,jaccard_coeff
0,0,0,1.000000
1,0,1,0.045977
2,0,2,0.025862
3,0,3,0.045977
4,0,4,0.025862


In [21]:
len(jaccard_coeff_between_nodes)

16313521

In [22]:
jaccard_coeff_between_nodes=jaccard_coeff_between_nodes[jaccard_coeff_between_nodes['source']!=jaccard_coeff_between_nodes['destination']]

In [23]:
fb_cudf.columns = ['source',	'destination']

In [24]:
fb_cudf['edgeflag']=1

In [25]:
jaccard_coeff_joined_with_edges = jaccard_coeff_between_nodes.merge(fb_cudf,on= ['source',	'destination'],how='left')


In [26]:
jaccard_coeff_joined_with_edges.head()

,source,destination,jaccard_coeff,edgeflag
0,13,2323,0.0,null
1,13,2324,0.0,null
2,13,2325,0.0,null
3,13,2326,0.0,null
4,13,2327,0.0,null


In [27]:
# We just want to see the jaccard coeff of new edges
new_edges_jaccard_coeff = jaccard_coeff_joined_with_edges[jaccard_coeff_joined_with_edges['edgeflag']!=1]

In [28]:
# Here are the predicted edges from our metric.
new_edges_jaccard_coeff.sort_values(by='jaccard_coeff',ascending=False).head(50)

,source,destination,jaccard_coeff,edgeflag
21288,15,335,1.0,null
22988,12,37,1.0,null
22994,12,43,1.0,null
26392,12,209,1.0,null
26393,12,210,1.0,null
26398,12,215,1.0,null
34961,12,74,1.0,null
36272,11,335,1.0,null
36854,12,335,1.0,null
37220,15,43,1.0,null


# 5. Basic Measures

In [29]:
print("Number of Nodes",fb_G.number_of_nodes())
print("Number of Edges",fb_G.number_of_edges())

Number of Nodes 4039
Number of Edges 176468


In [30]:
# Compute the indegree and outdegree to the node. In a directed graph this corresponds to no of followers and no of follows
fb_G.degrees().head()

,vertex,in_degree,out_degree
0,0,347,347
1,1,17,17
2,2,10,10
3,3,17,17
4,4,10,10


# Benchmarking

## A. On Facebook Data

In [49]:
# Creating graphs for Benchmarking using cuDF and networkx
# graphX
# Loading the file as cudf

fb_cudf = cudf.read_csv("facebook_combined.txt", sep=' ', names=['src', 'dst'],dtype =['int32','int32'])
fb_G = cugraph.Graph()
fb_G.add_edge_list(fb_cudf["src"],fb_cudf["dst"])

In [50]:
# reading the dataset
fb = nx.read_edgelist('facebook_combined.txt', create_using = nx.Graph(), nodetype = int)

### 1. Connected Components

In [72]:
%%timeit
ccs = []
for i, x in enumerate(nx.connected_components(fb)):
    ccs.append(x)

10.4 ms ± 109 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [73]:
%%timeit
ccs = cugraph.strongly_connected_components(fb_G)

29 ms ± 4.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 2. Shortest Path

In [74]:
%%timeit
nx.single_source_shortest_path(fb,1)

14.3 ms ± 22 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [75]:
%%timeit
distances = cugraph.sssp(fb_G, 1)

3.02 ms ± 7.55 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 3. Pagerank

In [79]:
%%timeit
pageranks = nx.pagerank(fb)

3.06 s ± 20.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [80]:
%%timeit
fb_pagerank = cugraph.pagerank(fb_G)

4.33 ms ± 46.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## B. On Twitter Data

In [102]:
# Creating graphs for Benchmarking using cuDF and networkx
# graphX
# Loading the file as cudf

twitter_cudf = cudf.read_csv("twitter_combined.txt", sep=' ', names=['src', 'dst'],dtype =['int32','int32'])
pd_twitter_df = twitter_cudf.to_pandas()

In [103]:
nodes = set(list(pd_twitter_df['src'].unique()) + list(pd_twitter_df['dst'].unique()))

In [104]:
nodes_int_dict = {}
for i,node in enumerate(nodes):
    nodes_int_dict[node]=i

In [105]:
# create a map of city and a unique id
pd_twitter_df['src_int'] = pd_twitter_df['src'].apply(lambda x : nodes_int_dict[x])
pd_twitter_df['dst_int'] = pd_twitter_df['dst'].apply(lambda x : nodes_int_dict[x])

In [110]:
twitter_cudf = cudf.from_pandas(pd_twitter_df)
twitter_cudf['src_int'] = twitter_cudf['src_int'].astype(np.int32)
twitter_cudf['dst_int'] = twitter_cudf['dst_int'].astype(np.int32)

In [111]:
twitter_G = cugraph.Graph()
twitter_G.add_edge_list(twitter_cudf["src_int"],twitter_cudf["dst_int"])

In [115]:
# reading the dataset
twitter = nx.read_edgelist('twitter_combined.txt', create_using = nx.Graph(), nodetype = int)

### 1. Connected Components

In [116]:
%%timeit
ccs = []
for i, x in enumerate(nx.connected_components(twitter)):
    ccs.append(x)

535 ms ± 2.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [117]:
%%timeit
ccs = cugraph.strongly_connected_components(twitter_G)

6.47 s ± 3.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2. Shortest Paths

In [121]:
%%timeit
nx.single_source_shortest_path(twitter,35389442)

582 ms ± 5.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [122]:
%%timeit
distances = cugraph.sssp(twitter_G, 1)

3.27 ms ± 586 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 3. Pagerank

In [124]:
%%time
pageranks = nx.pagerank(fb)

CPU times: user 34.8 s, sys: 1.06 s, total: 35.9 s
Wall time: 35.9 s


In [125]:
%%timeit
fb_pagerank = cugraph.pagerank(fb_G)

4.35 ms ± 22.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## C. Google Plus Data

In [147]:
pd_gplus_df = pd.read_csv("gplus_combined.txt", sep=' ', names=['src', 'dst'])

nodes = set(list(pd_gplus_df['src'].unique()) + list(pd_gplus_df['dst'].unique()))

nodes_int_dict = {}
for i,node in enumerate(nodes):
    nodes_int_dict[node]=i
    

pd_gplus_df['src_int'] = pd_gplus_df['src'].apply(lambda x : nodes_int_dict[x])
pd_gplus_df['dst_int'] = pd_gplus_df['dst'].apply(lambda x : nodes_int_dict[x])

gplus_cudf = cudf.from_pandas(pd_gplus_df)
gplus_cudf['src_int'] = gplus_cudf['src_int'].astype(np.int32)
gplus_cudf['dst_int'] = gplus_cudf['dst_int'].astype(np.int32)

gplus_G = cugraph.Graph()
gplus_G.add_edge_list(gplus_cudf["src_int"],gplus_cudf["dst_int"])

In [148]:
# reading the dataset
gplus = nx.read_edgelist('gplus_combined.txt', create_using = nx.Graph(), nodetype = int)

### 1. Connected Components

In [149]:
%%time
ccs = []
for i, x in enumerate(nx.connected_components(gplus)):
    ccs.append(x)

CPU times: user 5.17 s, sys: 129 ms, total: 5.3 s
Wall time: 5.3 s


In [150]:
%%time
ccs = cugraph.strongly_connected_components(gplus_G)

CPU times: user 6.42 s, sys: 21.9 s, total: 28.3 s
Wall time: 27.7 s


### 2. Shortest Paths

In [151]:
%%time
ssp = nx.single_source_shortest_path(gplus,109247306373593947755)

CPU times: user 6.96 s, sys: 420 ms, total: 7.38 s
Wall time: 7.38 s


In [155]:
%%time
ssp = cugraph.sssp(gplus_G, 0)

CPU times: user 47.1 ms, sys: 36.9 ms, total: 83.9 ms
Wall time: 81.2 ms


### 3. Pagerank

In [156]:
%%time
pageranks = nx.pagerank(gplus)

CPU times: user 5min 40s, sys: 22.7 s, total: 6min 2s
Wall time: 6min 3s


In [157]:
%%time
fb_pagerank = cugraph.pagerank(gplus_G)

CPU times: user 14.5 ms, sys: 27.6 ms, total: 42.1 ms
Wall time: 39 ms


## Results

Connected Components

In [159]:
results = pd.read_excel("results.xlsx")

In [160]:
results.head()

,Module,Algorithm,Dataset,Time
0,Networkx,Connected Components,FB,10.40
1,Networkx,Shortest Paths,FB,14.30
2,Networkx,Pagerank,FB,3000.00
3,cuGraph,Connected Components,FB,29.00
4,cuGraph,Shortest Paths,FB,3.02


In [172]:
px.bar(results,x ='Dataset',y='Time',color = 'Module',facet_row = 'Algorithm')